In [1]:
hf_api_key= 'YOU_API_KEY'

In [2]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain_community.llms import LlamaCpp
from langchain.chains import RetrievalQA, LLMChain
import pathlib
import textwrap
from IPython.display import display
from IPython.display import Markdown
from PyPDF2 import PdfReader

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
def get_pdf_text(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PdfReader(file)
        text = "".join(page.extract_text() for page in reader.pages if page.extract_text())
    return text

def get_text_chunks(text):
    splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    return splitter.split_text(text)


In [4]:
pdf_path='Diabetes-Mellitus-English.pdf'
raw_text = get_pdf_text(pdf_path)
text_chunks = get_text_chunks(raw_text)


In [5]:
print(text_chunks)
#print(docs[0])

['Diabetes Mellitus  \n1. What is diabetes?  \n\uf0b7 Diabetes is a disorder of metabolism where blood glucose (blood sugar) remains  \ntoo high due to insufficient insulin production or inadequate action of available \ninsulin.  \n2. What are the types of diabetes?  \n\uf0b7 Basically,  diabetes mellitus can be of two types, based on how they are \ntreated/managed. One that definitively requires insulin from the very beginning \nfor treatment (often called type 1 DM). The  other types are those that do not \nordinarily require insulin for treatment *. These types of diabetes are a \nheterogenous group with type 2 DM forming the major numbers (more than 90 \nper cent) . \n\uf0b7 (* All diabetics may require insulin for treatment in special s ituations).  \n \n\uf0b7 There are two other conditions:  \na) Pre-diabetes  is a condition when a person’s blood glucose levels are higher \nthan normal but not high enough to be called Type 2 DM . These blood sugar \nof such patients may go into 

In [6]:
from langchain.schema import Document
documents = [Document(page_content=chunk) for chunk in text_chunks]

In [7]:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")

/var/folders/4r/tdxq1ynn3gzcvknj7pgx3ml00000gn/T/ipykernel_42532/2576288788.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")


In [8]:
vecstore = Chroma.from_documents(documents,embeddings)
query = "How to check BSL while on treatment? With or without medicines?"
search = vecstore.similarity_search(query)
to_markdown(search[0].page_content)

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


> interactions. Secondly, these patients are very prone to two complications  
> a) Hypoglycemi a with hypoglycemic drugs and insulin  
> b) Dehydration due to hyperglycemia resulting in a life -
> threatening hyperosmolar state because of hypernatremia. 
> (increase in sodium)  
>  
> 39. What is the effect of stress (mental and physical) on BSL in normal persons 
> and in diabetic individuals?  
>  Physical stress, like exercise, will reduce blood sugar levels.  
>  Mental stress can increase secretion of cortisol and/or adrenaline to marginally 
> increase the blood sugar values, both in normal individuals as well as in 
> diabetics.  
>  Ther e are no major changes in blood sugar during stress of minor surgeries 
> and/or minor injuries. However, if there is a major injury (RTA) or/and if 
> individual has to undergo major surgeries, increase in blood sugar values are 
> anticipated and these times usua lly require insulin/or escalation of antidiabetic 
> treatment.  
>  
> 40. What is the effect of sleep on DM and vice versa?  
>  Adequate, restful sleep is part of a healthy life -style, essential for management of 
> all life -style diseases.  
>  However, diabetes can have a profo und effect on sleep. Low blood sugar 
> (hypoglycemia) occurring during sleep can cause nightmares and restlessness in 
> sleep. Such patients may also complain of early morning headaches.  
>  Uncontrolled hyperglycemia related metabolic changes can have a negative 
> effect on the quality of sleep, which can vary from insomnia to early morning 
> awakening. Further these patients may not feel fresh after getting up  
>  At times, high carbohydrate intake does cause drowsiness,  and laziness during 
> day time  
>  
>  
>  41. Is taking advice from dietitian mandatory? Any diet prescription required?  
>  Learning about and managing one’s diet is essential for management of diabetes. 
> However, a dietary prescription is not mandatory.  
>  Diet of any individual is not to be changed completely for treatment  of diabetes. 
> Changes should be suggested within the usual diet options for the patient, and 
> that also address the working hours of the patient.  
>  
> 42. How to manage diet and medicines while travelling?  
>  Diet and medicines are easily manageable during short trave ls. Longer travels 
> may require greater adjustments.  
>  Some things that may help include:  
> a) Avoid long gaps and/or missing meals  
> b) Ensure adequate hydration  
> c) Carry snacks with themselves at all times  
> d) Avoid taking insulin in case a meal is likely to be delayed for 
> unavoidable reason.  
>  
> 43. How to manage diabetes during prolonged fasting as during Navratri and/or 
> Ramadaan?  
>  Diabetics on insulin should not fast  
>  If possible, all diabetics should refrain from fa sting  
>  Type 2 diabetics on treatment should consult their physician while deciding on 
> the dose and timing of their drugs  
>  
> 44. Can I modify medicines when I happen to eat too much/little? How?  
>  Modification of doses of insulin can be done by an individual, based on pre -meal 
> and post -meal SMBG  
>  Modification of oral drugs is not required.  
>  
>  
>  45. How do other minor illnesses (infections, minor injuries, etc.) affect DM? What 
> precautions are needed?  
>  Minor illnesses and/or injuries do not affect diabetes mellitus, if its ad equately 
> managed or controlled.  
>  No precautions are required except that it is best to avoid injuries by being 
> careful, because even a minor injury on the foot can turn into serious diabetic 
> foot.  
>  
> 46. As I am on many restrictions my social life gets compromise d 
>  Ordinarily, there are not many restrictions, and diabetes mellitus does not affect 
> social life of any individual, it is mostly a perception that needs to be addressed 
> during a discussion with peers and one’s physician.  
>  
> 47. Can I stop medicine when my blood sugar is in normal range?  
>  It may be possible to reduce medication in a type 2 diabetic but generally not 
> possible in a patient using insulin.  
>  
> 48. What to do when I miss my dose of medicine?  
>  Missing an occasional dose is not detrimental but regularity of medication must 
> be ensured.  
>  
> 49. How to check BSL while on treatment? With or without medicines?  
>  Self-monitoring as well as laboratory testing is to be done with medicines.  
>  
> 50. Does it mean that I have serious condition if I am started on insulin? Is it true 
> that  once insulin is started, it will be there for lifelong?  
>  Type 1 diabetic will require insulin for life.  
>  Type 2 diabetics require insulin intermittently during periods such as stress, 
> infections, surgery etc. but can go back to their oral medication once pe riods of 
> stress are over.  
>  51. Does taking bitter guard juice help?  
>  It may be of help, but bitter gourd as a treatment for diabetes should not be 
> taken without consultation with an Ayush practitioner, as even such treatments 
> have side effects.

In [9]:
retriever = vecstore.as_retriever(search_kwargs={'k':5})
retriever.get_relevant_documents(query)

/var/folders/4r/tdxq1ynn3gzcvknj7pgx3ml00000gn/T/ipykernel_42532/999673874.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents(query)
Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


[Document(metadata={}, page_content='interactions. Secondly, these patients are very prone to two complications  \na) Hypoglycemi a with hypoglycemic drugs and insulin  \nb) Dehydration due to hyperglycemia resulting in a life -\nthreatening hyperosmolar state because of hypernatremia. \n(increase in sodium)  \n \n39. What is the effect of stress (mental and physical) on BSL in normal persons \nand in diabetic individuals?  \n\uf0b7 Physical stress, like exercise, will reduce blood sugar levels.  \n\uf0b7 Mental stress can increase secretion of cortisol and/or adrenaline to marginally \nincrease the blood sugar values, both in normal individuals as well as in \ndiabetics.  \n\uf0b7 Ther e are no major changes in blood sugar during stress of minor surgeries \nand/or minor injuries. However, if there is a major injury (RTA) or/and if \nindividual has to undergo major surgeries, increase in blood sugar values are \nanticipated and these times usua lly require insulin/or escalation of anti

In [10]:
llm = LlamaCpp(model_path='BioMistral-7B.Q2_K.gguf',temperature = 0.3,max_tokens =2048,top_=1)

/Users/ajaypillai/Desktop/venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3517: UserWarning: WARNING! top_ is not default parameter.
                top_ was transferred to model_kwargs.
                Please confirm that top_ is what you intended.
  if await self.run_code(code, result, async_=asy):
llama_model_load_from_file_impl: using device Metal (Apple M3) - 9799 MiB free
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from BioMistral-7B.Q2_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = hub
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32 

In [11]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

template = """
<|context|>
You are an AI assistant that follows instruction extremely well.
Please be truthful and give direct answers
</s>
<|user|>
{query}
</s>
 <|assistant|>
"""

prompt = ChatPromptTemplate.from_template(template)

rag_chain = (
    {'context': retriever, 'query': RunnablePassthrough()} | prompt| llm | StrOutputParser()
)

In [12]:
response = rag_chain.invoke( "what are the different types of diabetes")
to_markdown(response)

Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3
llama_perf_context_print:        load time =    3207.02 ms
llama_perf_context_print: prompt eval time =    3206.95 ms /    59 tokens (   54.36 ms per token,    18.40 tokens per second)
llama_perf_context_print:        eval time =   13318.94 ms /   201 runs   (   66.26 ms per token,    15.09 tokens per second)
llama_perf_context_print:       total time =   16575.04 ms /   260 tokens


> There are several types of diabetes, including type 1 diabetes (T1D), type 2 diabetes (T2D), gestational diabetes, and other forms of diabetes. T1D is an autoimmune disease that affects both children and adults. It is characterized by insulin deficiency and typically requires insulin therapy to control blood glucose levels. T2D is a non-autoimmune disease that can affect people of all ages. It can be controlled through lifestyle changes, such as diet and exercise, or medication. Gestational diabetes is a form of diabetes that occurs during pregnancy. It typically resolves after delivery but may increase the risk of developing T2D later in life. Other types of diabetes include prediabetes, which is a condition in which blood gllucose levels are higher than normal but not high enough to meet the criteria for diabetes; and insulin resistance, which is a condition in which body cells do not respond properly to insulin.

In [13]:
response = rag_chain.invoke( "what are the risk factors of diabetes")
to_markdown(response)

Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3
Llama.generate: 44 prefix-match hit, remaining 15 prompt tokens to eval
llama_perf_context_print:        load time =    3207.02 ms
llama_perf_context_print: prompt eval time =     697.67 ms /    15 tokens (   46.51 ms per token,    21.50 tokens per second)
llama_perf_context_print:        eval time =    4446.62 ms /    65 runs   (   68.41 ms per token,    14.62 tokens per second)
llama_perf_context_print:       total time =    5156.47 ms /    80 tokens


>  The risk factors of diabetes are: 1) Overweight or obese 2) Old age 3) Family history of type 2 diabetes 4) History of gestational diabetes 5) Physical inactivity 6) History of certain cancers (breast, colon, pancreatic, liver).